In [1]:
import sys
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/chao/cantera', '/home/chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import cantera as ct
print(ct.__file__)

import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity

/home/chao/cantera/build/python/cantera/__init__.py


In [3]:
path_to_cti = '/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_bm/cpox_no_cov_bm.yaml'
gas = ct.Solution(path_to_cti, 'gas')
surf = ct.Interface(path_to_cti, 'surface1', [gas])
species_dict = rmgpy.data.base.Database().get_species('/home/chao/sketches/cpox_sim/cpox_no_cov/species_dictionary.txt',resonance=False)

In [4]:
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)

In [5]:
def find_species_phase_index(gas, surf, species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i
    
def change_species_enthalpy(gas, surf, species_name, dH, temp):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(gas, surf, species_name)

    species = phase.species(index)
    print(f"Initial H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
#     print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    
def correct_binding_energy(surf, gas, target_surf, spec_index, center_atom, x, temperature):
    """
    Here applies linear scaling relationship to calculate binding energy (J/kmol) from one metal
    to another
    Args:
        target_surf: binding energy dictionary of interest eg: Pt_111
        spec_index: index of the species to change
        center_atom: center atom of the species to change
        x: bonds of center atoms of the species to change
    """
    pt_111 = {'H': (-2.75368,'eV/molecule'),
              'C': (-7.02516,'eV/molecule'),
              'N': (-4.63225,'eV/molecule'),
              'O': (-3.81153,'eV/molecule'),
             }
    x_max = {'H': 1,
             'C': 4,
             'N': 3,
             'O': 2,
            }
    deltaE = ((x_max[center_atom]-x) / x_max[center_atom]) * (target_surf[center_atom][0]-pt_111[center_atom][0])*9.6487e7
    print(f"\nApplying LSR correction to {surf.species(spec_index).name}:")
    print(f"Changing H by {deltaE/1e6:+.1f} kJ/mol   ({deltaE/96487000.:+.2f} eV)")
    change_species_enthalpy(gas, surf, surf.species(spec_index).name, deltaE, temperature)

In [6]:
inputs = [('H', 0), ('C', 4), ('C', 2), ('C', 4), ('O', 0), ('C', 2), ('C', 3), ('C', 1), ('C', 0), ('H', 1), ('O', 1),
           ('O', 2), ('C', 3), ('C', 3), ('C', 3), ('C', 1), ('C', 2), ('C', 1), ('C', 1)]

In [7]:
for i in range(1, surf.n_species):
    print(surf.species(i), inputs[i-1])

<Species HX(21)> ('H', 0)
<Species CO2X(22)> ('C', 4)
<Species COX(23)> ('C', 2)
<Species CH4X(24)> ('C', 4)
<Species OX(25)> ('O', 0)
<Species CH2X(26)> ('C', 2)
<Species CH3X(27)> ('C', 3)
<Species CHX(28)> ('C', 1)
<Species CX(29)> ('C', 0)
<Species H2X(30)> ('H', 1)
<Species OHX(31)> ('O', 1)
<Species H2OX(32)> ('O', 2)
<Species CHOX(33)> ('C', 3)
<Species CH3OX(61)> ('C', 3)
<Species HOCOX(138)> ('C', 3)
<Species CHOX(231)> ('C', 1)
<Species CH2OX(232)> ('C', 2)
<Species C2H3X(241)> ('C', 1)
<Species C2HOX(246)> ('C', 1)


In [8]:
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        print(f'Converting C={c_binding_energy}, O={o_binding_energy}')
        path_to_cti = '/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_bm/cpox_no_cov_bm.yaml'
        gas1 = ct.Solution(path_to_cti, 'gas')
        surf1 = ct.Interface(path_to_cti, 'surface1', [gas1])
        be_dic = {'C': (c_binding_energy, 'eV/molecule'),
                  'O': (o_binding_energy, 'eV/molecule'),
                  'H': (-2.75368,'eV/molecule')
                 }
        for i in range(1, surf1.n_species, 1):
            correct_binding_energy(surf1, gas1, be_dic, i, inputs[i-1][0], inputs[i-1][1], 700)
        data_root = '/home/chao/sketches/cpox_sim/cpox_no_cov_bm_surfs'
        if not os.path.exists(data_root):
            os.mkdir(data_root)  
#         gas.write_yaml(os.path.join(data_root, f'dm_cpox_c{c_binding_energy}_o{o_binding_energy}.yaml'))
        surf1.write_yaml(os.path.join(data_root, f'cpox_bm_no_cov_c{c_binding_energy}_o{o_binding_energy}.yaml'))
        

Converting C=-5.5, O=-3.25

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -191.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +54.2 kJ/mol   (+0.56 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -77.3 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 90.7 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 11.3 kJ/mol

Apply


Applying LSR correction to CH3OX(61):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -257.9 kJ/mol
Modified H(700) = -221.1 kJ/mol

Applying LSR correction to HOCOX(138):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -350.6 kJ/mol
Modified H(700) = -313.8 kJ/mol

Applying LSR correction to CHOX(231):
Changing H by +110.4 kJ/mol   (+1.14 eV)
Initial H(700) = -228.6 kJ/mol
Modified H(700) = -118.2 kJ/mol

Applying LSR correction to CH2OX(232):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = -188.1 kJ/mol
Modified H(700) = -114.5 kJ/mol

Applying LSR correction to C2H3X(241):
Changing H by +110.4 kJ/mol   (+1.14 eV)
Initial H(700) = -59.3 kJ/mol
Modified H(700) = 51.1 kJ/mol

Applying LSR correction to C2HOX(246):
Changing H by +110.4 kJ/mol   (+1.14 eV)
Initial H(700) = -222.3 kJ/mol
Modified H(700) = -112.0 kJ/mol
Converting C=-5.5, O=-4.5

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(70


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +61.5 kJ/mol   (+0.64 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -203.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +54.2 kJ/mol   (+0.56 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -77.3 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +61.5 kJ/mol   (+0.64 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 78.6 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +30.8 kJ/mol   (+0.32 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 5.3 kJ/mol

Applying LSR correction to CHX(28


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +61.5 kJ/mol   (+0.64 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -203.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by -66.4 kJ/mol   (-0.69 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -197.9 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +61.5 kJ/mol   (+0.64 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 78.6 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +30.8 kJ/mol   (+0.32 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 5.3 kJ/mol

Applying LSR correction to CHX(2

Changing H by +74.2 kJ/mol   (+0.77 eV)
Initial H(700) = -222.3 kJ/mol
Modified H(700) = -148.2 kJ/mol
Converting C=-6.0, O=-3.75

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +49.5 kJ/mol   (+0.51 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -215.6 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +5.9 kJ/mol   (+0.06 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -125.5 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +49.5 kJ/mol   (+0.51 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 66.6 kJ/mol

Applying LSR correction to CH3X(27):
Cha

...


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +1.2 kJ/mol   (+0.01 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -263.8 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by -66.4 kJ/mol   (-0.69 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -197.9 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +1.2 kJ/mol   (+0.01 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 18.3 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +0.6 kJ/mol   (+0.01 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -24.8 kJ/mol

Applying LSR correction to CHX(28

Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = -188.1 kJ/mol
Modified H(700) = -198.9 kJ/mol

Applying LSR correction to C2H3X(241):
Changing H by -16.3 kJ/mol   (-0.17 eV)
Initial H(700) = -59.3 kJ/mol
Modified H(700) = -75.5 kJ/mol

Applying LSR correction to C2HOX(246):
Changing H by -16.3 kJ/mol   (-0.17 eV)
Initial H(700) = -222.3 kJ/mol
Modified H(700) = -238.6 kJ/mol
Converting C=-7.25, O=-3.5

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -275.9 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(2


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -275.9 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by -66.4 kJ/mol   (-0.69 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -197.9 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 6.3 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by -5.4 kJ/mol   (-0.06 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -30.9 kJ/mol

Applying LSR correction to CHX(2


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -288.0 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +30.1 kJ/mol   (+0.31 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -101.4 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = -5.8 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by -11.5 kJ/mol   (-0.12 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -36.9 kJ/mol

Applying LSR correction to CHX

Converting C=-7.5, O=-5.0

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -288.0 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by -114.7 kJ/mol   (-1.19 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -246.2 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = -5.8 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by -11.5 kJ/mol   (-0.12 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -36.9 kJ/mol

App

In [9]:
energy_c = np.arange(-5.5, -7.75, -0.25)
energy_o = np.arange(-3.25, -5.5, -0.25)
with open('/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_bm/cpox_no_cov_bm.yaml', 'r') as f:
    model = yaml.load(f, Loader=yaml.FullLoader)
for i in energy_c:
    for j in energy_o:
        path_to_cti = f'/home/chao/sketches/cpox_sim/cpox_no_cov_bm_surfs/cpox_bm_no_cov_c{i}_o{j}.yaml'
        with open(path_to_cti, 'r') as f:
            model_to_change = yaml.load(f, Loader=yaml.FullLoader)

        model_to_change['elements'] = model['elements']

        with open(path_to_cti, 'w') as f:
            yaml.dump(model_to_change, f, default_flow_style=False)
            
        gas = ct.Solution(path_to_cti, 'gas')
        surf = ct.Interface(path_to_cti, 'surface1', [gas])